##### Flower project / Classification problem

<img src="./favicon.jpg" alt="flower image" />

###### Environment initialization

In [1]:
%tensorflow_version 2.x

UsageError: Line magic function `%tensorflow_version` not found.


In [3]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import pandas as pd

from IPython.display import clear_output


###### Dataset loading

In [4]:
#train_path = tf.keras.utils.get_file("iris_training.csv", "https://storage.googleapi.com/download.tensorflow.org/data/iris_training.csv")
#eval_path  = tf.keras.utils.get_file("iris_test.csv", "https://storage.googleapi.com/download.tensorflow.org/data/iris_test.csv")

CSV_COLUMN_NAME = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth', 'Species']
SPECIES = ['Setosa', 'Versicolor', 'Virginica']

df_train = pd.read_csv('iris_training.csv', names=CSV_COLUMN_NAME, header=0)
df_eval  = pd.read_csv('iris_test.csv', names=CSV_COLUMN_NAME, header=0)


In [5]:
df_train.head(5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [6]:
df_eval.head(5)

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,5.9,3.0,4.2,1.5,1
1,6.9,3.1,5.4,2.1,2
2,5.1,3.3,1.7,0.5,0
3,6.0,3.4,4.5,1.6,1
4,5.5,2.5,4.0,1.3,1


###### Dataset processing

In [7]:
y_train = df_train.pop('Species')
y_eval  = df_eval.pop('Species')

features_column = []
for key in df_train.keys():
    features_column.append(tf.feature_column.numeric_column(key=key))

print(features_column)    

[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


###### convert pandas dataset to tf.data.Dataset Object

In [8]:
def input_fn(features, labels, training=True, batch_size=256):
    # let convert input to tensforflow Dataset fromat
    ds = tf.data.Dataset.from_tensor_slices((dict(features), labels))
    
    # Shuffle if in training mode
    if training:
        ds = ds.shuffle(1000).repeat()
        
    return ds.batch(batch_size)

###### creating the model

In [9]:
classifier = tf.estimator.DNNClassifier(feature_columns=features_column, hidden_units=[30, 10], n_classes=3)
clear_output()

###### training the model

In [11]:
classifier.train(input_fn=lambda: input_fn(df_train, y_train, training=True), steps=5000)

# eval
eval_result = classifier.evaluate(input_fn=lambda: input_fn(df_eval, y_eval, training=False))
print('='*115)
print('\nTest set accuracy : {accuracy:0.3f}\n'.format(**eval_result))
print('='*115)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp32smevtv/model.ckpt-5000
Instructions for updating:
Use standard file utilities to get mtimes.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Calling checkpoint listeners before saving checkpoint 5000...
INFO:tensorflow:Saving checkpoints for 5000 into /tmp/tmp32smevtv/model.ckpt.
INFO:tensorflow:Calling checkpoint listeners after saving checkpoint 5000...
INFO:tensorflow:loss = 0.6245415, step = 5000
INFO:tensorflow:global_step/sec: 157.548
INFO:tensorflow:loss = 0.62999344, step = 5100 (0.638 sec)
INFO:tensorflow:global_step/sec: 172.281
INFO:tensorflow:loss = 0.60988635, step = 5200 (0.580 sec)
INFO:tensorflow:global_step/sec: 190.904
INFO:tensorflow:loss = 0.61308384, step = 5300 (0.524 sec)
INFO:tensorflow:global_step/sec: 192.53

###### make prediction

In [12]:
def prediction():
    def input_fn(features, batch_size=256):
        return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
    
    features = ['SepalLength', 'SepalWidth', 'PetalLength', 'PetalWidth']
    predict = {}
    
    print("Please type numeric value as prompted:")
    for feature in features:
        valid = True
        while valid:
            val = input(feature + ": ")
            if not val.isdigit(): valid = False
            
        predict[feature] = [float(val)]
        
    predictions = classifier.predict(input_fn=lambda: input_fn(predict))
    for prediction in predictions:
        print(prediction)
        class_id = prediction['class_ids'][0]
        probability = prediction['probabilities'][class_id]
        
        print('{}\nPrediction is "{}" ({:.1f}%)\n{}'.format('='*115, SPECIES[class_id], 100 * probability, '='*115))

In [13]:
prediction()

Please type numeric value as prompted:
SepalLength: 5.1
SepalWidth: 3.3
PetalLength: 1.7
PetalWidth: 0.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp32smevtv/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([ 1.7725794 ,  0.35830453, -0.8228857 ], dtype=float32), 'probabilities': array([0.75889057, 0.18448769, 0.0566217 ], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" (75.9%)


In [14]:
prediction()

Please type numeric value as prompted:
SepalLength: 5.9
SepalWidth: 3.0
PetalLength: 4.2
PetalWidth: 1.5
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp32smevtv/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([0.83265096, 2.629861  , 2.2061238 ], dtype=float32), 'probabilities': array([0.09105946, 0.54934293, 0.3595976 ], dtype=float32), 'class_ids': array([1]), 'classes': array([b'1'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Versicolor" (54.9%)


In [15]:
prediction()

Please type numeric value as prompted:
SepalLength: 6.9
SepalWidth: 3.1
PetalLength: 5.4
PetalWidth: 2.1
INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from /tmp/tmp32smevtv/model.ckpt-10000
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
{'logits': array([0.8719103, 3.2146893, 3.7371318], dtype=float32), 'probabilities': array([0.03452673, 0.3594276 , 0.60604566], dtype=float32), 'class_ids': array([2]), 'classes': array([b'2'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Virginica" (60.6%)


In [ ]:
#---------------------------------------------
#here wehave an test case with espected result
#| SPECIES     | SL   SW   PL   PW  |
#"Sotosa"     => 5.1  3.3  1.7  0.5 
#"Versicolor" => 5.9  3.0  4.2  1.5
#"Virginica"  => 6.9  3.1  5.4  2.1
#---------------------------------------------